In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.models.resnet import resnet18  # Using a basic, small ResNet variant
from torch import nn

# Transformation for CIFAR-10 images
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Standard normalization for CIFAR-10
])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

# Modify ResNet for CIFAR-10 (assume using resnet18)
model = resnet18()
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)  # Adjusting first conv layer to accept 32x32 input
model.maxpool = nn.Identity()  # Remove maxpool to keep dimensionality
model.fc = nn.Linear(model.fc.in_features, 10)  # Adjusting final fully connected layer for 10 classes

# Assuming use of a GPU if available
device = torch.device("mps")
model.to(device)

Files already downloaded and verified
Files already downloaded and verified


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): Identity()
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), p

In [2]:
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy of the model on the test images: {accuracy}%')
    return accuracy

In [ ]:
import torch.optim as optim
from torch import nn

# Set up the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# Define number of epochs
num_epochs = 1

for epoch in range(num_epochs):  # Loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # Get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # Print every 200 mini-batches
            print(f'Epoch {epoch + 1}, Batch {i + 1}: Loss: {running_loss / 200:.4f}')
            running_loss = 0.0

print('Finished Training')
torch.save(model.state_dict(), "state")

# Evaluate the model
evaluate_model(model, testloader)


In [65]:
model = resnet18()  # Not using pre-trained weights
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)  # Adjusting first conv layer to accept 32x32 input
model.maxpool = nn.Identity()  # Remove maxpool to keep dimensionality
model.fc = nn.Linear(model.fc.in_features, 10)  # Adjusting final fully connected layer for 10 classes
model.to(device)
model.load_state_dict(torch.load("state"))


<All keys matched successfully>

In [66]:
evaluate_model(model, testloader)


Accuracy of the model on the test images: 73.11%


In [27]:
%load_ext autoreload
%autoreload now
import copy
import pruning_funcs

percent_prune = []
percent_prune_with_bernoulli = []

for percent in range(0,15):
    pruned_model = copy.deepcopy(model)
    pruned_model_2 = copy.deepcopy(model)
    pruning_funcs.percent_prune(pruned_model, device, percent=percent)
    percent_prune.append(evaluate_model(pruned_model, testloader))
    pruning_funcs.percent_prune_with_bernulli(pruned_model_2, device, percent=percent)
    percent_prune_with_bernoulli.append(evaluate_model(pruned_model_2, testloader))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Accuracy of the model on the test images: 63.12%
Accuracy of the model on the test images: 69.04%
